In [1]:
# Initial Imports
import pandas as pd
import numpy as np
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy

In [2]:
# Read in lyrics data
lyrics = pd.read_csv("data/songdata.csv")

In [101]:
# Required columns
cols=['song', 'artist', 'trackid', 'danceability', 'energy', 'key', 'mode', 
      'speechiness', 'acousticness', 'instrumentalness', 'liveness', 
      'valence', 'tempo', 'duration_ms', 'time_signature',
     'duration', 'analysis_sample_rate', 'analysis_channels', 
      'end_of_fade_in', 'start_of_fade_out', 'loudness', 'tempo', 'tempo_confidence', 
      'time_signature', 'time_signature_confidence', 'key', 'key_confidence', 'mode', 'mode_confidence']

In [102]:
# Creating dataframe for audio features
features=pd.DataFrame(columns=cols)

In [103]:
artists = ['Eminem', 'Beatles', 'Train', 'Elton John', 'Michael Buble','Ella Fitzgerald','Foo Fighters', 'Radiohead', 'Red Hot Chili Peppers','John Legend','Katy Perry', 'Rihanna','Sam Smith','Sia', 'Whitney Houston','Owl City','Queen' ]

In [104]:
songs_selected=lyrics[lyrics.artist.isin(artists)]

In [105]:
sample=songs_selected.sample(200)

In [106]:
sample.reset_index(inplace=True)

In [107]:
# Client Credentials
client_id = "5fc0da4106eb48e1913868f4950828fb"
client_secret = "5a549bc4674e47dfaf3b06da4500a44d"

client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

for index,row in sample.iterrows():
    song = row['song']
    
    temp = pd.DataFrame(columns = cols)
    temp['song'] = [song]
    temp['artist'] = [row['artist']]
    
    try:
        result = sp.search(song, type='track', limit=1)

    except error as e:
        print(e)
        continue
        
    if len(result['tracks']['items']) == 0:
        temp['trackid'] = ['NA']
        features = features.append(temp, ignore_index=True)
        continue
    
    trackid=result['tracks']['items'][0]['id']
    temp['trackid'] = [trackid]
    
    try:
        feat=sp.audio_features(trackid)[0]
        analysis=sp.audio_analysis(trackid)['track']
    except error as e:
        print(e)
        continue
    
    combined = {**feat, **analysis}
    
    for col in cols:
        if col in ['song', 'artist', 'trackid']:
            continue
        temp[col]= combined[col]
        
    features = features.append(temp, ignore_index=True)

In [108]:
features.to_csv('feautures200.csv', index=None)

In [109]:
features.head()

,song,artist,trackid,danceability,energy,key,mode,speechiness,acousticness,instrumentalness,...,start_of_fade_out,loudness,tempo,tempo_confidence,time_signature,time_signature_confidence,key,key_confidence,mode,mode_confidence
0,Please Baby Don't,John Legend,5WUgOOaTk00PnksDekcgcg,0.720,0.705,5,1,0.0408,0.506000,0.000001,...,155.84653,-6.820,106.332,0.844,4,1.000,5,0.046,1,0.383
1,Milk,Foo Fighters,0hfByzDRgR10HDLqm89lA9,0.461,0.869,1,0,0.0731,0.000616,0.000008,...,164.14766,-4.672,145.919,0.378,4,1.000,1,0.000,0,0.276
2,Early Autumn,Ella Fitzgerald,0Eb4LOT2fnm0PHuLyGbHoN,0.768,0.165,9,1,0.0958,0.329000,0.845000,...,130.93732,-11.712,77.992,0.357,4,0.866,9,0.221,1,0.347
3,I Just Don't Feel Like Christmas,Rihanna,7y8I1aTNxww4JgN1duLWyf,0.663,0.476,0,1,0.0366,0.361000,0.000000,...,219.13252,-10.653,126.224,0.379,4,0.996,0,0.707,1,0.696
4,Crab Man,Ella Fitzgerald,0K8wwzUxwACiuYZhfCZ8Xb,0.665,0.754,11,1,0.0510,0.004530,0.000116,...,198.39710,-5.832,134.990,0.470,4,0.806,11,0.000,1,0.057


In [9]:
import nltk
nltk.download('punkt')
from nltk.tokenize import RegexpTokenizer

all_text = ' '.join(lyrics.text)
all_text = all_text.replace('\n', '')
sent_text = nltk.sent_tokenize(all_text)

[nltk_data] Downloading package punkt to /Users/user/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [10]:
tokenizer = RegexpTokenizer(r'\w+').tokenize

sent_tokenized = []
for sent in sent_text:
    sent_tokenized.append(tokenizer(sent))

In [11]:
from gensim.models import Word2Vec
from gensim.test.utils import common_texts, get_tmpfile

path = get_tmpfile("word2vec.model")
model = Word2Vec(sent_tokenized, size=200, window=5, min_count=3, workers=4)

model.save("word2vec.model")

In [110]:
wv = []
for lyr in sample.text:
    vec = [0] * 200
    c = 0
    words = tokenizer(lyr)
    for word in words:
        try:
            vec = vec + model[word]
            c = c + 1
        except:
            vec = vec + [0] * 200
    vec = np.array(vec) / c
    wv.append(vec)

/Applications/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


In [111]:
req_cols = list(range(200))
wv_df = pd.DataFrame(wv[:1000])[req_cols]
wv_df['song'] = sample['song']

In [112]:
wv_df.to_csv('word_vectors_songs.csv', index=None)

In [113]:
features.head()

,song,artist,trackid,danceability,energy,key,mode,speechiness,acousticness,instrumentalness,...,start_of_fade_out,loudness,tempo,tempo_confidence,time_signature,time_signature_confidence,key,key_confidence,mode,mode_confidence
0,Please Baby Don't,John Legend,5WUgOOaTk00PnksDekcgcg,0.720,0.705,5,1,0.0408,0.506000,0.000001,...,155.84653,-6.820,106.332,0.844,4,1.000,5,0.046,1,0.383
1,Milk,Foo Fighters,0hfByzDRgR10HDLqm89lA9,0.461,0.869,1,0,0.0731,0.000616,0.000008,...,164.14766,-4.672,145.919,0.378,4,1.000,1,0.000,0,0.276
2,Early Autumn,Ella Fitzgerald,0Eb4LOT2fnm0PHuLyGbHoN,0.768,0.165,9,1,0.0958,0.329000,0.845000,...,130.93732,-11.712,77.992,0.357,4,0.866,9,0.221,1,0.347
3,I Just Don't Feel Like Christmas,Rihanna,7y8I1aTNxww4JgN1duLWyf,0.663,0.476,0,1,0.0366,0.361000,0.000000,...,219.13252,-10.653,126.224,0.379,4,0.996,0,0.707,1,0.696
4,Crab Man,Ella Fitzgerald,0K8wwzUxwACiuYZhfCZ8Xb,0.665,0.754,11,1,0.0510,0.004530,0.000116,...,198.39710,-5.832,134.990,0.470,4,0.806,11,0.000,1,0.057


In [114]:
# Removing empty rows
features = features[features['trackid'] != 'NA']

In [115]:
# Making a copy for preservation
features_copy = features.copy()

In [116]:
wv_df.head()

,0,1,2,3,4,5,6,7,8,9,...,191,192,193,194,195,196,197,198,199,song
0,-0.382913,-0.082338,-0.062737,0.070332,-0.725707,-0.225276,-0.353025,0.202337,-0.094829,-0.951479,...,-0.422009,0.609671,0.664388,0.074438,-0.157670,0.234941,0.579265,0.876113,-0.531330,Please Baby Don't
1,-0.014819,-0.103624,-0.038024,0.302434,-0.138430,-0.342051,-0.253609,0.115050,-0.098961,-0.087977,...,-0.379614,0.091470,0.461595,-0.055310,-0.087324,0.354566,-0.355873,0.299554,0.221939,Milk
2,-0.123901,-0.029310,-0.118937,0.285294,-0.521666,-0.339652,-0.314318,0.174780,-0.178619,-0.433396,...,-0.502615,0.186782,0.556150,0.152642,-0.288623,0.536218,-0.178414,0.534236,-0.442307,Early Autumn
3,-0.061755,0.083784,-0.139761,0.055425,-0.319134,-0.320648,-0.293192,0.065888,-0.459359,-0.877977,...,-0.397818,0.603455,0.443926,0.065484,0.002968,1.010506,-0.129806,0.787325,-0.457692,I Just Don't Feel Like Christmas
4,-0.317295,-0.221647,0.131035,-0.154978,-0.067846,-0.304391,-0.602074,-0.254899,0.516035,-0.303304,...,-0.469601,-0.276018,-0.102966,-0.387280,0.044553,-0.176696,0.266139,0.324368,0.299969,Crab Man


In [222]:
df_all=pd.merge(features, wv_df, how='left', on='song')

In [223]:
cols=list(df_all.columns)

In [224]:
#cols.remove('level_0')
cols.remove('index')

ValueError: list.remove(x): x not in list

In [225]:
# Remove columns for clustering
cols.remove('song')
cols.remove('artist')
cols.remove('trackid')


In [226]:
fcols = list(features.columns)

In [227]:
cols.remove('song')
cols.remove('artist')
cols.remove('trackid')

ValueError: list.remove(x): x not in list

In [228]:
fcols.remove('song')
fcols.remove('artist')
fcols.remove('trackid')

In [229]:
df_all=df_all.dropna()

In [230]:
df_all.shape

(200, 230)

In [231]:
from sklearn import preprocessing

x = features[fcols].values #returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
df = pd.DataFrame(x_scaled)
df['song']=features['song']
df['artist']=features['artist']

/Applications/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype object was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)


In [232]:
dfcols=list(df.columns)
dfcols.remove('song')
dfcols.remove('artist')

In [233]:
# Kmeans
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=10, random_state=0).fit(df[dfcols])
df['label'] = kmeans.labels_

In [234]:
df.label.unique()

array([8, 1, 2, 7, 5, 0, 3, 6, 4, 9])

In [235]:
from sklearn.cluster import AgglomerativeClustering

cluster = AgglomerativeClustering(n_clusters=20, affinity='euclidean', linkage='ward')  
cluster.fit_predict(df[dfcols])

df.label=cluster.labels_

In [236]:
df[df.artist=='Rihanna']

,0,1,2,3,4,5,6,7,8,9,...,28,29,30,31,32,33,34,song,artist,label
3,0.701511,0.474210,0.000000,0.000000,1.0,1.0,0.013760,0.362811,0.000000,0.274709,...,0.000000,0.000000,0.707,1.0,1.0,0.696,0.777778,I Just Don't Feel Like Christmas,Rihanna,2
11,0.607053,0.669994,0.000000,0.000000,1.0,1.0,0.025263,0.010447,0.000003,0.460756,...,0.000000,0.000000,0.422,1.0,1.0,0.479,0.777778,Electricity,Rihanna,2
35,0.234257,0.824400,0.636364,0.636364,1.0,1.0,0.018168,0.011754,0.000081,0.194767,...,0.636364,0.636364,0.640,1.0,1.0,0.633,0.555556,Here I Go Again,Rihanna,3
49,0.535264,0.577148,0.000000,0.000000,1.0,1.0,0.009460,0.004226,0.000358,0.322674,...,0.000000,0.000000,0.824,1.0,1.0,0.594,0.777778,We All Want Love,Rihanna,2
57,0.637280,0.517605,0.818182,0.818182,1.0,1.0,0.016448,0.480399,0.000000,0.244186,...,0.818182,0.818182,0.313,1.0,1.0,0.433,0.555556,Written In The Stars,Rihanna,13
60,1.000000,0.652837,0.090909,0.090909,0.0,0.0,0.285100,0.094970,0.000000,0.736919,...,0.090909,0.090909,0.362,0.0,0.0,0.546,0.333333,Hard,Rihanna,6
64,0.512594,0.592286,0.818182,0.818182,1.0,1.0,0.047732,0.001502,0.000000,0.104070,...,0.818182,0.818182,0.622,1.0,1.0,0.476,0.555556,Silly Boy,Rihanna,18
66,0.736776,0.399530,0.727273,0.727273,1.0,1.0,0.038809,0.706531,0.000000,0.642442,...,0.727273,0.727273,0.494,1.0,1.0,0.622,0.888889,The One,Rihanna,0
79,0.782116,0.593295,1.000000,1.000000,0.0,0.0,0.072028,0.013261,0.000000,0.013227,...,1.000000,1.000000,0.388,0.0,0.0,0.541,0.111111,Dem Haters,Rihanna,11
128,0.486146,0.972752,0.181818,0.181818,1.0,1.0,0.040636,0.000032,0.000000,0.194767,...,0.181818,0.181818,0.685,1.0,1.0,0.505,0.777778,Do Ya Thing,Rihanna,2


In [237]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

sentiment_data = pd.DataFrame(columns=['positive', 'negative', 'neutral', 'compound', 'song'])
for index, row in sample.iterrows():
    pos = []
    neg = []
    neu = []
    compound = []
    sentences=nltk.sent_tokenize(row.text)
    analyzer = SentimentIntensityAnalyzer()
    for sentence in sentences:
        vs = analyzer.polarity_scores(sentence)
        pos.append(vs['pos'])
        neg.append(vs['neg'])
        neu.append(vs['neu'])
        compound.append(vs['compound'])
    tmp = pd.DataFrame([[np.average(pos), np.average(neg), np.average(neu), np.average(compound), row['song']]], columns=['positive', 'negative', 'neutral', 'compound', 'song'])
    sentiment_data = pd.concat([sentiment_data, tmp], ignore_index=True)

In [238]:
df_all_with_sentiment=pd.merge(df_all, sentiment_data, how='left', on='song') 

In [239]:
df_with_sentiment = pd.merge(df, sentiment_data, how='left', on='song')

In [241]:
df_with_sentiment=df_with_sentiment.dropna()

In [242]:
x = df_with_sentiment[dfcols + ['positive', 'negative', 'neutral', 'compound']].values #returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
df_with_sentiment = pd.DataFrame(x_scaled, columns=dfcols + ['positive', 'negative', 'neutral', 'compound'])
df_with_sentiment['song']=features['song']
df_with_sentiment['artist']=features['artist']

In [243]:
from sklearn.cluster import AgglomerativeClustering
# df=df.dropna()
cluster = AgglomerativeClustering(n_clusters=20, affinity='euclidean', linkage='ward')  
cluster.fit_predict(df_with_sentiment[dfcols + ['positive', 'negative', 'neutral', 'compound']])

df_with_sentiment.label=cluster.labels_

/Applications/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  


In [246]:
df[df.label==2]

,0,1,2,3,4,5,6,7,8,9,...,28,29,30,31,32,33,34,song,artist,label
3,0.701511,0.474210,0.000000,0.000000,1.0,1.0,0.013760,0.362811,0.000000,0.274709,...,0.000000,0.000000,0.707,1.0,1.0,0.696,0.777778,I Just Don't Feel Like Christmas,Rihanna,2
11,0.607053,0.669994,0.000000,0.000000,1.0,1.0,0.025263,0.010447,0.000003,0.460756,...,0.000000,0.000000,0.422,1.0,1.0,0.479,0.777778,Electricity,Rihanna,2
17,0.455919,0.489348,0.090909,0.090909,1.0,1.0,0.196947,0.172860,0.000000,0.172965,...,0.090909,0.090909,0.753,1.0,1.0,0.639,0.777778,Breaking Hearts,Elton John,2
26,0.680101,0.634672,0.090909,0.090909,1.0,1.0,0.011288,0.006598,0.000000,0.053634,...,0.090909,0.090909,0.113,1.0,1.0,0.350,0.777778,If We Ever Meet Again,Katy Perry,2
48,0.754408,0.570083,0.000000,0.000000,1.0,1.0,0.006558,0.020196,0.000000,0.103779,...,0.000000,0.000000,0.923,1.0,1.0,0.822,0.777778,Kiss Me Babe It's Christmas Time,Owl City,2
49,0.535264,0.577148,0.000000,0.000000,1.0,1.0,0.009460,0.004226,0.000358,0.322674,...,0.000000,0.000000,0.824,1.0,1.0,0.594,0.777778,We All Want Love,Rihanna,2
62,0.555416,0.483293,0.000000,0.000000,1.0,1.0,0.018706,0.451253,0.001693,0.148256,...,0.000000,0.000000,0.970,1.0,1.0,0.869,0.777778,Dressing Up,Katy Perry,2
65,0.607053,0.441916,0.000000,0.000000,1.0,1.0,0.007525,0.317584,0.000003,0.199128,...,0.000000,0.000000,0.802,1.0,1.0,0.723,0.777778,Heaven For Everyone,Queen,2
69,0.760705,0.801189,0.000000,0.000000,1.0,1.0,0.016986,0.196981,0.011111,0.453488,...,0.000000,0.000000,0.759,1.0,1.0,0.690,0.777778,The Invisible Man,Queen,2
72,0.494962,0.803207,0.000000,0.000000,1.0,1.0,0.012470,0.237182,0.000000,0.120640,...,0.000000,0.000000,0.638,1.0,1.0,0.719,0.777778,Peppermint Winter,Owl City,2
